In [3]:
! pip install contractions
! pip install punktab

ERROR: Could not find a version that satisfies the requirement punktab (from versions: none)
ERROR: No matching distribution found for punktab


In [5]:
import pandas as pd
import numpy as np
import contractions
import re
import contractions
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.util import ngrams

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
df = pd.read_csv('/Users/jorgejimenez/Documents/UP/NLP/RegresionLogistica/deepsea.csv')
df.head()

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
category_mapping = {'biology': 0, 'meme': 1}
data_df['label'] = data_df['category'].map(category_mapping)
data_df.head()

In [ ]:
def process_text(text):
    # Arreglar contracciones
    text = contractions.fix(text)
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres que no sean letras
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenizar
    tokens = word_tokenize(text)
    # Quitar stopwords
    tokens = [token for token in tokens if token not in stop_words]
    # Lematizar cada token
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Reconstruir el texto
    return ' '.join(tokens)


In [ ]:
tqdm.pandas()
data_df['clean_text'] = data_df['tweet'].progress_apply(process_text)
data_df.head()


In [ ]:
def generate_ngrams(n, text_series):
    ngram_list = []
    for text in text_series:
        # Generar tokens y sus n-gramas
        tokens = word_tokenize(text)
        n_grams = ngrams(tokens, n)
        for gram in n_grams:
            ngram_list.append(' '.join(gram))
    return pd.Series(ngram_list).value_counts()


In [ ]:

# N-gramas (bigrams) para todos los tweets
n = 2
bigram_freq = generate_ngrams(n, data_df['clean_text']).head(30)
print(bigram_freq.head(10))


In [ ]:
plt.figure(figsize=(8,6))
plt.barh(bigram_freq.index, bigram_freq.values)
plt.xlabel('Frecuencia')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
wordcloud_all = WordCloud(width=2000, height=2000, colormap='Wistia_r',
                          background_color='black', min_font_size=8)
wordcloud_all = wordcloud_all.generate_from_frequencies(bigram_freq)
plt.figure(figsize=(5,5))
plt.imshow(wordcloud_all)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
bigram_meme = generate_ngrams(n, data_df[data_df['category'] == 'meme']['clean_text']).head(30)
plt.figure(figsize=(8,6))
plt.barh(bigram_meme.index, bigram_meme.values)
plt.xlabel('Frecuencia')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:

wordcloud_meme = WordCloud(width=2000, height=2000, colormap='Wistia_r',
                           background_color='black', min_font_size=8)
wordcloud_meme = wordcloud_meme.generate_from_frequencies(bigram_meme)
plt.figure(figsize=(5,5))
plt.imshow(wordcloud_meme)
plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# Bigrams para tweets de la categoría "biology"
bigram_biology = generate_ngrams(n, data_df[data_df['category'] == 'biology']['clean_text']).head(30)
wordcloud_biology = WordCloud(width=2000, height=2000, colormap='Wistia_r',
                              background_color='black', min_font_size=8)
wordcloud_biology = wordcloud_biology.generate_from_frequencies(bigram_biology)
plt.figure(figsize=(5, 5))
plt.imshow(wordcloud_biology)
plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = tfidf_vectorizer.fit_transform(data_df['clean_text'])
tfidf_matrix


In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, init='random')
tsne_coords = tsne.fit_transform(tfidf_matrix)
data_df['tsne_x'] = tsne_coords[:, 0]
data_df['tsne_y'] = tsne_coords[:, 1]


In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(nube)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
 # ngram with biology only
 n = 2
cym_ngrams = generate_ngrams(n, df[df['category'] == 'biology']['clean_tweet']).head(30)
cym_ngrams.head(10)

In [ ]:
nube=WordCloud(width=2000, height=2000, colormap='Wistia_r', background_color='black',min_font_size=8).generate_from_frequencies(cym_ngrams)

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(nube)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tdidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorized_tweets = tdidf_vectorizer.fit_transform(df['clean_tweet'])
vectorized_tweets

In [ ]:
from sklearn.manifold import TSNE
model = TSNE(n_components=2, init='random')
tsne_result = model.fit_transform(vectorized_tweets)
tsne_result

In [ ]:
df['tsne_1'] = tsne_result[:,0]
df['tsne_2'] = tsne_result[:,1]

In [ ]:
import plotly.express as px
fig = px.scatter(
    data_frame=df,
    x=df['tsne_1'],
    y=df['tsne_2'],
    color=df['etiqueta'],
    template='plotly_dark',
    hover_data=['tweet']
)
fig.show()

In [ ]:
from sklearn.model_selection import train_test_split
X = df['clean_tweet']
Y = df['etiqueta']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, random_state=101)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1,2))

#Fit transform en datos de entrenamiento
X_train_vectorized = tfidf.fit_transform(X_train)

#Transform en datos de prueba
X_test_vectorized = tfidf.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_vectorized, Y_train)

In [ ]:
y_pred = model.predict(X_test_vectorized)

In [ ]:
y_pred

In [ ]:
Y_test

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
sns.heatmap(confusion_matrix(Y_test,y_pred), annot=True, fmt='.2f')

In [ ]:
print(classification_report(Y_test,y_pred))
# precision me importan los falsos positivos
# recall me importan los falsos negativos
# f1-score me importa el overall de todo